In [1]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
from torchvision import datasets, models, transforms

In [2]:
import os
import PIL.Image as Image

print(os.listdir("./Dog/Images"))

# with Image.open("./Dog/Images/n02085620-Chihuahua/n02085620_368.jpg") as im:
#     print(im.size)


['n02097658-silky_terrier', 'n02092002-Scottish_deerhound', 'n02099849-Chesapeake_Bay_retriever', 'n02091244-Ibizan_hound', 'n02095314-wire-haired_fox_terrier', 'n02091831-Saluki', 'n02102318-cocker_spaniel', 'n02104365-schipperke', 'n02090622-borzoi', 'n02113023-Pembroke', 'n02105505-komondor', 'n02093256-Staffordshire_bullterrier', 'n02113799-standard_poodle', 'n02109961-Eskimo_dog', 'n02089973-English_foxhound', 'n02099601-golden_retriever', 'n02095889-Sealyham_terrier', 'n02085782-Japanese_spaniel', 'n02097047-miniature_schnauzer', 'n02110063-malamute', 'n02105162-malinois', 'n02086079-Pekinese', 'n02097130-giant_schnauzer', 'n02113978-Mexican_hairless', 'n02107142-Doberman', 'n02097209-standard_schnauzer', 'n02115913-dhole', 'n02106662-German_shepherd', 'n02106382-Bouvier_des_Flandres', 'n02110185-Siberian_husky', 'n02094258-Norwich_terrier', 'n02093991-Irish_terrier', 'n02094114-Norfolk_terrier', 'n02109525-Saint_Bernard', 'n02093754-Border_terrier', 'n02105251-briard', 'n0210855

In [3]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

dir = "./Dog/Images"
dog_dir_list = os.listdir(dir)
dog_dir_list = dog_dir_list[:10]

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

X = []
labels = []
targets = {}
class_index = -1
def training_data(label, data_dir):
    print('正在读入:' + data_dir)
    for img in os.listdir(data_dir):
        path = os.path.join(data_dir, img)
        im = Image.open(path).convert('RGB')
        image = transform(im)
        X.append(image)
        labels.append(label)

for dog_dir in dog_dir_list:
    label = dog_dir.split("-")[1]
    if label not in targets:
        targets[label] = class_index + 1
    training_data(targets[label], dir + "/" + dog_dir)
    class_index += 1

data = torch.stack(X)
labels = torch.LongTensor(labels)
print(data.shape)

正在读入:./Dog/Images/n02097658-silky_terrier
正在读入:./Dog/Images/n02092002-Scottish_deerhound
正在读入:./Dog/Images/n02099849-Chesapeake_Bay_retriever
正在读入:./Dog/Images/n02091244-Ibizan_hound
正在读入:./Dog/Images/n02095314-wire-haired_fox_terrier
正在读入:./Dog/Images/n02091831-Saluki
正在读入:./Dog/Images/n02102318-cocker_spaniel
正在读入:./Dog/Images/n02104365-schipperke
正在读入:./Dog/Images/n02090622-borzoi
正在读入:./Dog/Images/n02113023-Pembroke
torch.Size([1772, 3, 150, 150])


In [4]:
labels, targets

(tensor([0, 0, 0,  ..., 9, 9, 9]),
 {'silky_terrier': 0,
  'Scottish_deerhound': 1,
  'Chesapeake_Bay_retriever': 2,
  'Ibizan_hound': 3,
  'wire': 4,
  'Saluki': 5,
  'cocker_spaniel': 6,
  'schipperke': 7,
  'borzoi': 8,
  'Pembroke': 9})

In [5]:
class DogNet(nn.Module):
    def __init__(self):
        super(DogNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(
                in_channels=32,
                out_channels=64,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(
                in_channels=64,
                out_channels=128,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),
        )
        self.conv4 = nn.Sequential(
            nn.Conv2d(
                in_channels=128,
                out_channels=128,
                kernel_size=3,
            ),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Dropout(0.5),
        )
        self.fc1 = nn.Linear(in_features=6272,out_features=512)
        self.fc2 = nn.Linear(in_features=512,out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [86]:

model = DogNet()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

model.train()
for epoch in range(10):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()
    print(f"loss: {loss.item()}")



loss: 2.304170608520508
loss: 2.307943820953369
loss: 2.2896671295166016
loss: 2.2821996212005615
loss: 2.2774016857147217
loss: 2.2743887901306152
loss: 2.265643835067749
loss: 2.2581799030303955
loss: 2.2480573654174805
loss: 2.2447099685668945


In [87]:
model.eval()
y_pred = model(data)
y_pred = torch.argmax(y_pred, dim=1)
ok = (y_pred == labels).sum()
print(ok/len(y_pred))

tensor(0.1309)
